In [1]:
using ITensors, ITensorMPS
using LinearAlgebra
# using SparseArrays
using Plots
using JLD2

In [2]:
include("../src/main.jl")
include("../src/observables.jl")
include("../src/states.jl")
include("../src/doubled_measurements.jl")
include("../src/singled_measurements.jl")
include("../src/circuits.jl")

pure_sample (generic function with 1 method)

In [3]:
function forced_circuit(L::Int, T::Int, λ::Float64, δ::Float64; ref=true, PBC=false, final_perfect=false, observables=Symbol[], cutoff=1E-8, maxdim=200)
    ψ, sites = pure_ghz(L; ref=ref)

    data = Dict([s => zeros(ComplexF64, 2T+2) for s in observables])
    data = MPS_update_data(ψ, observables, data, 1; ref=ref, max_t=2T+2)
    data = MPS_update_data(ψ, observables, data, 2; ref=ref, max_t=2T+2)

    λzz = δ*(1-λ)
    λx = δ*λ

    for t in 1:T
        if !PBC
            ψ, _, _ = singled_measure(ψ, kron(PauliZ,PauliZ), λzz, 1:L-1, [false for _ in 1:L-1]; ref=ref)
        else
            ψ, _, _ = singled_measure(ψ, kron(PauliZ,PauliZ), λzz, 1:L; ref=ref)
        end
        ψ /= norm(ψ)
        truncate!(ψ; cutoff=cutoff, maxdim=maxdim)

        data = MPS_update_data(ψ, observables, data, 2t+1; ref=ref, max_t=2T+2)

        ψ, _, _ = singled_measure(ψ, PauliX, λx, 1:L, [false for _ in 1:L]; ref=ref)
        ψ /= norm(ψ)
        truncate!(ψ; cutoff=cutoff, maxdim=maxdim)
        # println(expect(ψ, "X"))

        data = MPS_update_data(ψ, observables, data, 2t+2; ref=ref, max_t=2T+2)
    end

    return ψ, data
end

forced_circuit (generic function with 1 method)